In [28]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.discrete.count_model import ZeroInflatedNegativeBinomialP

import warnings
warnings.filterwarnings('ignore')

##### modelo_bneg_direto

In [29]:
# Fisman, R.; Miguel, E. Corruption, Norms, and Legal Enforcement: Evidence
#from Diplomatic Parking Tickets.
# Journal of Political Economy, v. 15, n. 6, p. 1020-1048, 2007.
# https://www.journals.uchicago.edu/doi/abs/10.1086/527495

df_corruption = pd.read_csv('data/corruption.csv', delimiter=',')

In [30]:
# Modo direto de estimação do modelo binomial negativo:
    
modelo_bneg_direto = sm.NegativeBinomial.from_formula('violations ~ staff + post + corruption',
                                                      data=df_corruption).fit()

print(modelo_bneg_direto.summary())

Optimization terminated successfully.
         Current function value: 1.904031
         Iterations: 19
         Function evaluations: 23
         Gradient evaluations: 23
                     NegativeBinomial Regression Results                      
Dep. Variable:             violations   No. Observations:                  298
Model:               NegativeBinomial   Df Residuals:                      294
Method:                           MLE   Df Model:                            3
Date:                Thu, 26 Sep 2024   Pseudo R-squ.:                  0.1549
Time:                        21:20:43   Log-Likelihood:                -567.40
converged:                       True   LL-Null:                       -671.37
Covariance Type:            nonrobust   LLR p-value:                 8.088e-45
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       1.9469      0.205   

Alpha é estatísticamente significante (P>|z| < 0.05). Isso é um diagnóstico de superdispersão dos dados (cauda longa)!

In [31]:
# Teste de razão de verossimilhança
-2*(-2071.79 - (-567.401))

3008.7780000000002

In [32]:
# Cálculo manual do fit do 'modelo_bneg' antes da vigência da lei

modelo_bneg_direto.params

np.exp(1.946890 - 4.274634*0 + 0.040018*23 + 0.452654*0.5)

22.05702179606651

##### modelo_bneg_direto2

In [33]:
# Gerando novo dataframe ('df_corruption2') com seleção das variáveis originais
df_corruption2 = df_corruption.iloc[:, 0:6]

# Seleção das observações com 'violations' menores ou iguais a 3
df_corruption2 = df_corruption2[df_corruption2['violations'] <= 3]

In [34]:
# Modo direto de estimação do modelo binomial negativo 2:
    
modelo_bneg_direto2 = sm.NegativeBinomial.from_formula('violations ~ staff + post + corruption',
                                                      data=df_corruption2).fit()

print(modelo_bneg_direto2.summary())

Optimization terminated successfully.
         Current function value: 0.800565
         Iterations: 21
         Function evaluations: 25
         Gradient evaluations: 25
                     NegativeBinomial Regression Results                      
Dep. Variable:             violations   No. Observations:                  224
Model:               NegativeBinomial   Df Residuals:                      220
Method:                           MLE   Df Model:                            3
Date:                Thu, 26 Sep 2024   Pseudo R-squ.:                  0.1809
Time:                        21:20:43   Log-Likelihood:                -179.33
converged:                       True   LL-Null:                       -218.94
Covariance Type:            nonrobust   LLR p-value:                 4.484e-17
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.1241      0.152   

Alpha NÃO é estatísticamente significante (P>|z| > 0.05). Isso é um diagnóstico de AUSÊNCIA de superdispersão dos dados (ou presença de equidisperção)!

In [35]:
modelo_bneg_direto2.params

# Cálculo manual do fit do 'modelo_bneg_direto2' antes da vigência da lei
# igual à lousinha

(1 - (1/(1 + np.exp(-(-1.611649 - 0.952315*0.5)))))*\
    (np.exp(2.488877 + 0.020020*23 + 0.093722*0.5 - 4.287916*0))

17.802536661347645

##### modelo_zinb

In [36]:
# Estimação do modelo ZINB pela função 'ZeroInflatedNegativeBinomialP' do pacote
#'statsmodels.discrete.count_model'

# Definição da variável dependente (voltando ao dataset 'df_corruption')
y = df_corruption['violations']

# Definição das variáveis preditoras que entrarão no componente de contagem
x1 = df_corruption[['staff','post','corruption']]
X1 = sm.add_constant(x1)

# Dummização da variável 'post'
# Se estimarmos o modelo sem dummizar as variáveis categóricas, o modelo retorna
#um erro
X1 = pd.get_dummies(X1, columns=['post'], dtype=int, drop_first=True)

# Definição das variáveis preditoras que entrarão no componente logit (inflate)
x2 = df_corruption[['corruption']]
X2 = sm.add_constant(x2)

# O argumento 'exog_infl' corresponde às variáveis que entram no componente
#logit (inflate)
modelo_zinb = ZeroInflatedNegativeBinomialP(y, X1, exog_infl=X2,
                                            inflation='logit').fit()

         Current function value: 1.870015
         Iterations: 35
         Function evaluations: 39
         Gradient evaluations: 39


In [37]:
modelo_zinb.params

inflate_const        -17.985682
inflate_corruption    -8.110426
const                  2.032397
staff                  0.041076
corruption             0.181459
post_yes              -4.263797
alpha                  1.856016
dtype: float64

In [38]:
# Cálculo manual do fit do 'modelo_zinb' antes da vigência da lei

(1 - (1/(1 + np.exp(-(-17.985682 - 8.110426*0.5)))))*\
    (np.exp(2.032397 + 0.041076*23 + 0.181459*0.5 - 4.263797*0))

21.49616398112116